# **KAN**

In [3]:
import pandas as pd
import torch
from kan import KAN
from kan.utils import create_dataset, ex_round

# Step 1: Load the Dataset
df = pd.read_csv('datasets/numcubes.csv')
x = torch.tensor(df[['number']].values, dtype=torch.float32)
y = torch.tensor(df[['cube']].values, dtype=torch.float32)

# Create the dataset dictionary
dataset = {
    'train_input': x,
    'train_label': y,
    'test_input': x,   # Using the same data for simplicity in testing accuracy
    'test_label': y
}

# Step 2: Initialize the Model with only one neuron (input to output)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
din = 1
dout = 1
G = 4
K = 4
model = KAN(width=[1, 1], grid=G, k=K, seed=0, device=device)  # Single input-output neuron

# FLOP calculation function
def calculate_kan_flops(din, dout, G, K):
    return (din * dout) * (9 * K * (G + 1.5 * K) + 2 * G - 2.5 * K - 1)

# Calculate FLOPs for this KAN model
flops = calculate_kan_flops(din, dout, G, K)
print(f"Estimated FLOPs for the KAN model: {flops}")

# Visualize the initial model
model(dataset['train_input'])
model.plot()

# Step 3: Train the Model
# Training for 500 steps with Adam optimizer and lambda regularization
model.fit(dataset, opt="Adam", steps=500, lamb=0.0001)

# Visualize the trained model
model.plot()

# Step 4: Evaluate the Model (Mean Squared Error)
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    predictions = model(dataset['test_input'])
    mse_loss = torch.nn.functional.mse_loss(predictions, dataset['test_label'])
    print(f"\nMean Squared Error on Test Data: {mse_loss.item():.4f}")

# Step 5: Display the Learned Symbolic Formula
symbolic_formula = ex_round(model.symbolic_formula()[0][0], 4)
print("Learned symbolic formula:", symbolic_formula)

ImportError: cannot import name 'KAN' from 'kan' (unknown location)

# **MLP**

In [7]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error

# Step 1: Load the Dataset
df = pd.read_csv('datasets/numcubes.csv')
x = torch.tensor(df[['number']].values, dtype=torch.float32)
y = torch.tensor(df[['cube']].values, dtype=torch.float32)

# Step 2: Define the MLP Model with Customizable Parameters
class MLP(nn.Module):
    def __init__(self, input_size=1, hidden_size=10, output_size=1, num_hidden_layers=1):
        super(MLP, self).__init__()
        layers = []
        
        # Input layer
        layers.append(nn.Linear(input_size, hidden_size))
        layers.append(nn.ReLU())
        
        # Hidden layers
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(nn.ReLU())
        
        # Output layer
        layers.append(nn.Linear(hidden_size, output_size))
        
        # Create the sequential model
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

# Initialize the model with desired parameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 1
hidden_size = 10000
output_size = 1
num_hidden_layers = 1
model = MLP(input_size, hidden_size, output_size, num_hidden_layers).to(device)

# Move data to the device
x, y = x.to(device), y.to(device)

# Calculate FLOPs manually
def calculate_flops(model, input_size):
    flops = 0
    for layer in model.model:
        if isinstance(layer, nn.Linear):
            layer_flops = 2 * layer.in_features * layer.out_features
            flops += layer_flops
    return flops

# Display the calculated FLOPs for this model
flops = calculate_flops(model, input_size)
print(f"Estimated FLOPs for the MLP model: {flops}")

# Step 3: Define Loss Function and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Step 4: Train the Model
epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()               # Clear gradients
    predictions = model(x)              # Forward pass
    loss = criterion(predictions, y)    # Calculate loss
    loss.backward()                     # Backward pass
    optimizer.step()                    # Update weights

    if (epoch+1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Step 5: Evaluate the Model (Mean Squared Error)
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    predictions = model(x)
    mse_loss = criterion(predictions, y)
    print(f"\nMean Squared Error on Test Data: {mse_loss.item():.4f}")

# Convert predictions to numpy for easy comparison
predictions_np = predictions.cpu().numpy()
y_np = y.cpu().numpy()

# Display Predictions vs Actual Values
print("\nPredictions vs Actual:")
for i in range(len(y_np)):
    print(f"Number: {x[i].item()}, Predicted Square: {predictions_np[i][0]:.2f}, Actual Square: {y_np[i][0]}")


Estimated FLOPs for the MLP model: 40000
Epoch [100/500], Loss: 133952.3594
Epoch [200/500], Loss: 60397.2852
Epoch [300/500], Loss: 16213.2070
Epoch [400/500], Loss: 5709.9429
Epoch [500/500], Loss: 2616.2048

Mean Squared Error on Test Data: 2599.5886

Predictions vs Actual:
Number: 1.0, Predicted Square: -30.60, Actual Square: 1.0
Number: 2.0, Predicted Square: -5.53, Actual Square: 8.0
Number: 3.0, Predicted Square: 29.62, Actual Square: 27.0
Number: 4.0, Predicted Square: 78.97, Actual Square: 64.0
Number: 5.0, Predicted Square: 137.90, Actual Square: 125.0
Number: 6.0, Predicted Square: 226.13, Actual Square: 216.0
Number: 7.0, Predicted Square: 323.64, Actual Square: 343.0
Number: 8.0, Predicted Square: 497.51, Actual Square: 512.0
Number: 9.0, Predicted Square: 742.09, Actual Square: 729.0
Number: 10.0, Predicted Square: 998.82, Actual Square: 1000.0
Number: 11.0, Predicted Square: 1337.95, Actual Square: 1331.0
Number: 12.0, Predicted Square: 1809.37, Actual Square: 1728.0
Num